In [2]:
import pandas as pd
import os

In [9]:
df_path = "data/NU_340297544_01AGO2023_31AGO2023.csv"
df = pd.read_csv(df_path, sep=",", encoding="latin-1")
df = df.rename({'DescriÃÂ§ÃÂ£o': 'Descricao'}, axis = 1)
df.head()

,Data,Valor,Identificador,Descricao
0,03/08/2023,-4.0,64cb88e6-f81f-4b43-b9f8-dc1dc8f7d379,Compra no dÃÂ©bito - Suco Tal
1,03/08/2023,-9.9,64cbc2f3-9d0d-4795-8af0-b6910dc280ed,Compra no dÃÂ©bito - Mc Donalds
2,03/08/2023,-3.5,64cc3384-dde7-492f-b0e7-dab258bd4417,Compra no dÃÂ©bito - Dellicata Paes e Tenta
3,04/08/2023,800.0,64cd05e2-bd32-4023-bc2e-b94b77507cc6,TransferÃÂªncia recebida pelo Pix - WYNK SOLU...
4,04/08/2023,-2.5,64cd1991-67bf-4908-9804-ba7d5fa79446,TransferÃÂªncia enviada pelo Pix - Samuel Mot...


In [13]:
# tratar acentos nas linhas abaixo
for col in ['Descricao']:
    print(df[col])

0                          Compra no dAAbito - Suco Tal
1                        Compra no dAAbito - Mc Donalds
2            Compra no dAAbito - Dellicata Paes e Tenta
3     TransferAAancia recebida pelo Pix - WYNK SOLUC...
4     TransferAAancia enviada pelo Pix - Samuel Mota...
5                                   Pagamento de fatura
6            Compra no dAAbito - Dellicata Paes e Tenta
7     TransferAAancia enviada pelo Pix - Nilza Paes ...
8     TransferAAancia enviada pelo Pix - Samuel Mota...
9                                       AplicaAAAAo RDB
10    TransferAAancia enviada pelo Pix - Manuela Fon...
11    TransferAAancia enviada pelo Pix - PHOENIX INV...
12    TransferAAancia enviada pelo Pix - ANTONIO FLA...
13           Compra no dAAbito - Dellicata Paes e Tenta
14                     Compra no dAAbito - Sacolao Xepa
15    TransferAAancia enviada pelo Pix - Micaela Duq...
16    TransferAAancia recebida pelo Pix - TECWAY CON...
17    TransferAAancia enviada pelo Pix - Shopee 